Keras Recurrent Neural Nets

In [1]:
import load

In [2]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import Recurrent
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

from keras.layers.recurrent import SimpleRNN

from keras.layers.embeddings import Embedding
#import backend as K?

import matplotlib.pyplot as plt
plt.ion()

Using Theano backend.


In [30]:
from tqdm import tqdm

In [3]:
import pickle
train, test, dicts = pickle.load(open("atis.pkl"))

In [4]:
dicts.keys()

['labels2idx', 'tables2idx', 'words2idx']

In [5]:
dicts.values()

[{'B-aircraft_code': 0,
  'B-airline_code': 1,
  'B-airline_name': 2,
  'B-airport_code': 3,
  'B-airport_name': 4,
  'B-arrive_date.date_relative': 5,
  'B-arrive_date.day_name': 6,
  'B-arrive_date.day_number': 7,
  'B-arrive_date.month_name': 8,
  'B-arrive_date.today_relative': 9,
  'B-arrive_time.end_time': 10,
  'B-arrive_time.period_mod': 11,
  'B-arrive_time.period_of_day': 12,
  'B-arrive_time.start_time': 13,
  'B-arrive_time.time': 14,
  'B-arrive_time.time_relative': 15,
  'B-booking_class': 16,
  'B-city_name': 17,
  'B-class_type': 18,
  'B-compartment': 19,
  'B-connect': 20,
  'B-cost_relative': 21,
  'B-day_name': 22,
  'B-day_number': 23,
  'B-days_code': 24,
  'B-depart_date.date_relative': 25,
  'B-depart_date.day_name': 26,
  'B-depart_date.day_number': 27,
  'B-depart_date.month_name': 28,
  'B-depart_date.today_relative': 29,
  'B-depart_date.year': 30,
  'B-depart_time.end_time': 31,
  'B-depart_time.period_mod': 32,
  'B-depart_time.period_of_day': 33,
  'B-dep

In [6]:
train[2][0]

array([126, 126, 126, 126, 126,  48, 126,  35,  99, 126, 126, 126,  78,
       126,  14, 126, 126,  12], dtype=int32)

In [7]:
dicts[b'words2idx'] #gives index for words, we want opposite dict

{"'d": 0,
 "'ll": 1,
 "'m": 2,
 "'re": 3,
 "'s": 4,
 "'t": 5,
 '72s': 6,
 '<UNK>': 7,
 'DIGIT': 8,
 'DIGITDIGIT': 9,
 'DIGITDIGITDIGIT': 10,
 'DIGITDIGITDIGITDIGIT': 11,
 'DIGITDIGITDIGITDIGITDIGITDIGIT': 12,
 'a': 13,
 'abbreviation': 14,
 'abbreviations': 15,
 'about': 16,
 'ac': 17,
 'actually': 18,
 'after': 19,
 'afternoon': 20,
 'again': 21,
 'air': 22,
 'aircraft': 23,
 'airfare': 24,
 'airline': 25,
 'airlines': 26,
 'airplane': 27,
 'airplanes': 28,
 'airport': 29,
 'airports': 30,
 'alaska': 31,
 'all': 32,
 'along': 33,
 'also': 34,
 'am': 35,
 'america': 36,
 'american': 37,
 'amount': 38,
 'an': 39,
 'and': 40,
 'angeles': 41,
 'another': 42,
 'any': 43,
 'anywhere': 44,
 'ap': 45,
 'ap57': 46,
 'ap80': 47,
 'approximately': 48,
 'april': 49,
 'are': 50,
 'area': 51,
 'arizona': 52,
 'around': 53,
 'arrange': 54,
 'arrangements': 55,
 'arrival': 56,
 'arrivals': 57,
 'arrive': 58,
 'arrives': 59,
 'arriving': 60,
 'as': 61,
 'at': 62,
 'atl': 63,
 'atlanta': 64,
 'august':

In [8]:
#inversting the dictionary

idx2words = dict((v,k) for k, v in dicts[b'words2idx'].items())

In [9]:
idx2words

{0: "'d",
 1: "'ll",
 2: "'m",
 3: "'re",
 4: "'s",
 5: "'t",
 6: '72s',
 7: '<UNK>',
 8: 'DIGIT',
 9: 'DIGITDIGIT',
 10: 'DIGITDIGITDIGIT',
 11: 'DIGITDIGITDIGITDIGIT',
 12: 'DIGITDIGITDIGITDIGITDIGITDIGIT',
 13: 'a',
 14: 'abbreviation',
 15: 'abbreviations',
 16: 'about',
 17: 'ac',
 18: 'actually',
 19: 'after',
 20: 'afternoon',
 21: 'again',
 22: 'air',
 23: 'aircraft',
 24: 'airfare',
 25: 'airline',
 26: 'airlines',
 27: 'airplane',
 28: 'airplanes',
 29: 'airport',
 30: 'airports',
 31: 'alaska',
 32: 'all',
 33: 'along',
 34: 'also',
 35: 'am',
 36: 'america',
 37: 'american',
 38: 'amount',
 39: 'an',
 40: 'and',
 41: 'angeles',
 42: 'another',
 43: 'any',
 44: 'anywhere',
 45: 'ap',
 46: 'ap57',
 47: 'ap80',
 48: 'approximately',
 49: 'april',
 50: 'are',
 51: 'area',
 52: 'arizona',
 53: 'around',
 54: 'arrange',
 55: 'arrangements',
 56: 'arrival',
 57: 'arrivals',
 58: 'arrive',
 59: 'arrives',
 60: 'arriving',
 61: 'as',
 62: 'at',
 63: 'atl',
 64: 'atlanta',
 65: 'augu

In [10]:
#inversting the dictionary

idx2labels = dict((v,k) for k, v in dicts[b'labels2idx'].items())

In [11]:
idx2labels

{0: 'B-aircraft_code',
 1: 'B-airline_code',
 2: 'B-airline_name',
 3: 'B-airport_code',
 4: 'B-airport_name',
 5: 'B-arrive_date.date_relative',
 6: 'B-arrive_date.day_name',
 7: 'B-arrive_date.day_number',
 8: 'B-arrive_date.month_name',
 9: 'B-arrive_date.today_relative',
 10: 'B-arrive_time.end_time',
 11: 'B-arrive_time.period_mod',
 12: 'B-arrive_time.period_of_day',
 13: 'B-arrive_time.start_time',
 14: 'B-arrive_time.time',
 15: 'B-arrive_time.time_relative',
 16: 'B-booking_class',
 17: 'B-city_name',
 18: 'B-class_type',
 19: 'B-compartment',
 20: 'B-connect',
 21: 'B-cost_relative',
 22: 'B-day_name',
 23: 'B-day_number',
 24: 'B-days_code',
 25: 'B-depart_date.date_relative',
 26: 'B-depart_date.day_name',
 27: 'B-depart_date.day_number',
 28: 'B-depart_date.month_name',
 29: 'B-depart_date.today_relative',
 30: 'B-depart_date.year',
 31: 'B-depart_time.end_time',
 32: 'B-depart_time.period_mod',
 33: 'B-depart_time.period_of_day',
 34: 'B-depart_time.start_time',
 35: 'B-d

In [12]:
list(map(lambda x: idx2words[x], train[0][0]))

['i',
 'want',
 'to',
 'fly',
 'from',
 'boston',
 'at',
 'DIGITDIGITDIGIT',
 'am',
 'and',
 'arrive',
 'in',
 'denver',
 'at',
 'DIGITDIGITDIGITDIGIT',
 'in',
 'the',
 'morning']

In [17]:
w =5

In [18]:
longest = np.max(list(map(len, train[0])))

In [19]:
nlabels = len(idx2labels)

In [20]:
def contextwin(l, win):
    '''
    win :: int corresponding to the size of the window
    given a list of indexes composing a sentence

    l :: array containing the word indexes

    it will return a list of list of indexes corresponding
    to context windows surrounding each word in the sentence
    '''
    assert (win % 2) == 1
    assert win >= 1
    l = list(l)

    lpadded = win // 2 * [-1] + l + win // 2 * [-1]
    out = [lpadded[i:(i + win)] for i in range(len(l))]

    assert len(out) == len(l)
    return out

In [31]:
#from tqdm import tqdm #for progress bars

train_w = [contextwin(t,w) for t in tqdm(train[0])] 
#train_w = [contextwin(t,w) for t in train[0]] 
test_w = [contextwin(t,w) for t in test[0]]

train_hot = [np_utils.to_categorical(t, nlabels) for t in train[2]] 
test_hot = [np_utils.to_categorical(t, nlabels) for t in test[2]]

100%|██████████| 4978/4978 [00:00<00:00, 46101.12it/s]


In [22]:
contextwin(train[0][0],5)

[[-1, -1, 232, 542, 502],
 [-1, 232, 542, 502, 196],
 [232, 542, 502, 196, 208],
 [542, 502, 196, 208, 77],
 [502, 196, 208, 77, 62],
 [196, 208, 77, 62, 10],
 [208, 77, 62, 10, 35],
 [77, 62, 10, 35, 40],
 [62, 10, 35, 40, 58],
 [10, 35, 40, 58, 234],
 [35, 40, 58, 234, 137],
 [40, 58, 234, 137, 62],
 [58, 234, 137, 62, 11],
 [234, 137, 62, 11, 234],
 [137, 62, 11, 234, 481],
 [62, 11, 234, 481, 321],
 [11, 234, 481, 321, -1],
 [234, 481, 321, -1, -1]]

In [23]:
train_w[0]

[[-1, -1, 232, 542, 502],
 [-1, 232, 542, 502, 196],
 [232, 542, 502, 196, 208],
 [542, 502, 196, 208, 77],
 [502, 196, 208, 77, 62],
 [196, 208, 77, 62, 10],
 [208, 77, 62, 10, 35],
 [77, 62, 10, 35, 40],
 [62, 10, 35, 40, 58],
 [10, 35, 40, 58, 234],
 [35, 40, 58, 234, 137],
 [40, 58, 234, 137, 62],
 [58, 234, 137, 62, 11],
 [234, 137, 62, 11, 234],
 [137, 62, 11, 234, 481],
 [62, 11, 234, 481, 321],
 [11, 234, 481, 321, -1],
 [234, 481, 321, -1, -1]]

In [24]:
model = Sequential()
model.add(Embedding(len(idx2words)+1, 1024, input_length = w))
model.add(SimpleRNN(32))
#model.add(Activation('sigmoid'))
model.add(Dense(nlabels))
model.add(Activation('softmax'))
sgd = SGD()

In [25]:
list(map(len, train[0]))

[18,
 11,
 15,
 6,
 10,
 9,
 13,
 11,
 6,
 9,
 9,
 13,
 6,
 9,
 7,
 10,
 7,
 14,
 18,
 14,
 12,
 9,
 12,
 9,
 16,
 11,
 7,
 20,
 3,
 11,
 11,
 16,
 6,
 17,
 9,
 7,
 8,
 11,
 11,
 15,
 9,
 24,
 8,
 11,
 3,
 4,
 8,
 5,
 18,
 11,
 18,
 5,
 12,
 9,
 11,
 5,
 13,
 19,
 10,
 16,
 16,
 14,
 10,
 12,
 7,
 13,
 18,
 8,
 9,
 8,
 6,
 11,
 7,
 6,
 13,
 7,
 23,
 12,
 10,
 15,
 7,
 7,
 11,
 15,
 11,
 11,
 14,
 15,
 13,
 14,
 8,
 9,
 9,
 12,
 12,
 5,
 13,
 16,
 6,
 12,
 9,
 11,
 13,
 11,
 7,
 11,
 7,
 9,
 18,
 11,
 6,
 10,
 9,
 16,
 8,
 10,
 9,
 15,
 13,
 11,
 14,
 10,
 15,
 21,
 9,
 27,
 10,
 9,
 16,
 14,
 11,
 11,
 7,
 20,
 11,
 11,
 8,
 15,
 9,
 11,
 11,
 18,
 5,
 16,
 11,
 7,
 9,
 20,
 16,
 11,
 8,
 18,
 14,
 11,
 8,
 11,
 9,
 9,
 11,
 7,
 11,
 25,
 11,
 8,
 10,
 11,
 9,
 14,
 8,
 7,
 10,
 5,
 14,
 15,
 13,
 19,
 15,
 10,
 13,
 10,
 11,
 12,
 14,
 11,
 9,
 8,
 7,
 12,
 7,
 18,
 5,
 11,
 17,
 14,
 18,
 5,
 10,
 19,
 11,
 15,
 7,
 18,
 12,
 3,
 4,
 13,
 12,
 12,
 12,
 7,
 14,
 6,
 12,
 12,
 7,
 14,

In [26]:
%time model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

CPU times: user 14.1 ms, sys: 3.49 ms, total: 17.6 ms
Wall time: 15.6 ms


In [27]:
#h = model.fit(train_w[0], train_hot[0], batch_size =128, nb_epoch=3, show_accuracy = True, validation_data = (X_test, Y_test), verbose=1)

NameError: name 'X_test' is not defined

https://github.com/fchollet/keras/blob/master/examples/babi_rnn.py

In [33]:
for e in range(10):
    print (e)
    for i, (tr, hot) in tqdm(enumerate(zip(train_w, train_hot)), total=len(train_w)):
        h = model.fit(tr, hot, batch_size =1, nb_epoch=1, verbose=0)

  0%|          | 0/4978 [00:00<?, ?it/s]

0


 41%|████      | 2031/4978 [10:36<19:13,  2.55it/s]


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
#Taking so much time!^^^

In [35]:
#How many times we get it wrong
errs = np.sum([len(np.nonzero(model.predict_classes(tw)-t)[0])for tw, t in zip(test_w, test[2])])
                  
num_words = np.sum(list(map(len, test[2])))


10/10 [==============================] - 0s


0

In [38]:
float(errs) / (num_words)                

0.1146988475755599

In [39]:
h.history #from where is this coming/]?

{'acc': [1.0], 'loss': [0.020374027753859991]}

In [40]:
model.predict(test_w[0])

array([[  4.05125064e-07,   8.79171466e-07,   1.84009957e-06, ...,
          3.91406701e-07,   3.62535502e-07,   9.99838352e-01],
       [  4.48486134e-07,   1.06250695e-06,   1.96808242e-06, ...,
          4.99243811e-07,   3.96045124e-07,   9.99864280e-01],
       [  6.09029087e-07,   1.13197268e-06,   1.18475227e-05, ...,
          5.57940268e-07,   5.56270948e-07,   9.99798417e-01],
       ..., 
       [  1.54865756e-06,   4.36413575e-06,   3.16866499e-05, ...,
          1.22419669e-06,   1.10268149e-06,   9.99542534e-01],
       [  2.04087250e-04,   1.01793499e-03,   4.98081092e-03, ...,
          2.02778290e-04,   1.11742323e-04,   7.51283646e-01],
       [  1.94805179e-04,   9.29704809e-04,   4.53559384e-02, ...,
          2.13010833e-04,   1.05345083e-04,   9.14370269e-03]], dtype=float32)

In [41]:
model.predict_classes(test_w[0])

19/19 [==============================] - 0s


array([126, 126, 126, 126, 126, 126, 126, 126,  48, 126,  78, 123, 126,
       126, 126, 126, 126, 126, 123])

In [42]:
test[2][0]

array([126, 126, 126, 126, 126, 126, 126, 126,  48, 126,  78, 123, 126,
       126, 126, 126, 126,  71, 119], dtype=int32)